### Import modules

In [1]:
from scrapper.art_hash import get_hash
from scrapper.extract_artcls_data import current_artcls, artcl_data, artcl_content
from scrapper.location_detector import detect_location, validate_location, get_location_id
from scrapper.relevance_score_claculator import calculate_score as score
from database_manager.database_connector import dbconnection as db
from database_manager import querys

### Save url containing articles

In [ ]:
# Base url
news_url= "https://www.elnuevodia.com/noticias/seguridad"

### Construct data dictionary and insert data into database

In [ ]:
# Get the current articles in endi.com 
# under the Seguridad category
articles = current_artcls(news_url)

# Iterate over current articles
for article in articles:
    # Re-establish database connection
    db.reconnect()
    # Create communication cursor object
    cursor = db.cursor(buffered=True)

    # Get article hash
    art_hash = get_hash(article=article)
    # Fetch stored articles hashes
    cursor.execute(querys.FETCH_HASHES)
    # Select rows where article hash exists
    cursor.execute(querys.CHECK_HASH, [art_hash])
    # Count number of rows hash appears
    check = cursor.rowcount

    # Get article data and store to database only if
    # article hash do not exists; check variable must
    # be 0
    if check == 0:
        # Create data dictionary with extracted article initial
        # data and hash 
        # URL, Headline, Author, and Hash 
        query_dict = artcl_data(article=article)
        
        # Update data dictionary to store content data
        # Subheadline, Datetime, and Content
        query_dict.update(artcl_content(article_data=query_dict))
        
        # Insert article data into database table Articles 
        cursor.execute(querys.INSERT_ARTICLE, query_dict)
        # Commit the insert
        db.commit()

        # Get ID of inserted article
        art_id = cursor.lastrowid

        # Create list with article detected location 
        locs_found = detect_location(article_data=query_dict)
        # Create list with detected locations validated
        loc_validated = validate_location(location=locs_found)
        
        # Insert into ArticleLocationRelationship only if
        # valudad location contains values 
        if loc_validated:
            # Iterate over validated locations
            for loc in loc_validated:
                # Get the location id
                loc_id = get_location_id(validated_location=loc)
                
                # Insert ArticleID and LocationID into ArticlesLocationRelation
                query_dict = {"articleid": art_id, "locationid": loc_id}
                cursor.execute(querys.INSERT_ARTICLE_LOCATION, query_dict)
                db.commit()

                # Calculate relevance score for the given location
                sc = score(location_id=loc_id)
                # Update the location relevance score
                query_dict = {"locationid":loc_id , "score": sc}
                cursor.execute(querys.UPDATE_RELEVACE_SCORE, query_dict)
                db.commit()
    cursor.close()
    db.close()

## Print Articles rows

In [2]:
db.reconnect()
query = """SELECT * FROM Articles"""
cursor = db.cursor(buffered=True)
cursor.execute(query)
for row in cursor:
    print(row)
db.close()

(1, 'https://www.elnuevodia.com/noticias/seguridad/notas/arrestan-a-cinco-presuntos-miembros-de-una-ganga-sospechosa-de-cometer-crimenes-violentos-en-santa-isabel-y-coamo/', 'Arrestan a cinco presuntos miembros de una ganga sospechosa de cometer crímenes violentos en Santa Isabel y Coamo', 'Por Alex Figueroa Cancel', 'La Policía indicó que los detenidos figuran entre las edades de 19 a 23 años', datetime.datetime(2024, 7, 25, 13, 42), 'ad1d7d3dbba78796329c59cf7c5b725c7f533ce1478c0609ab95e5b4cc6bd010')
(2, 'https://www.elnuevodia.com/noticias/seguridad/notas/investigan-si-cuerpo-hallado-en-rio-grande-corresponde-a-victima-de-un-hit-and-run/', 'Investigan si cuerpo hallado en Río Grande corresponde a víctima de un “hit and run”', 'Por Alex Figueroa Cancel', 'Un ciudadano alegó a la Policía que vio cuando fue atropellado', datetime.datetime(2024, 7, 25, 6, 23), '37f843f48093b4b0e8ad60f9b4f9ddb9999cf029dbad5c54e971d5f42339fc51')
(3, 'https://www.elnuevodia.com/noticias/seguridad/notas/roba

## Print Articles-Location Relation rows

In [ ]:
db.reconnect()
query = """SELECT * FROM ArticlesLocationRelation"""
cursor = db.cursor(buffered=True)
cursor.execute(query)
for row in cursor:
    print(row)
db.close()

## Print Location rows

In [ ]:
db.reconnect()
query = """SELECT * FROM Location"""
cursor = db.cursor(buffered=True)
cursor.execute(query)
for row in cursor:
    print(row)
db.close()

# Testing